In [1]:
import warnings
warnings.filterwarnings("ignore")

import env
import wrangle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# modeling methods
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression 

np.set_printoptions(suppress=True)

In [2]:
df = pd.read_csv("clean_zillow.csv")

In [3]:
train, validate, test = wrangle.split_zillow(df)

In [4]:
train = wrangle.prep_longitude_latitude_houseage_clusters(train)
validate = wrangle.prep_longitude_latitude_houseage_clusters(validate)
test = wrangle.prep_longitude_latitude_houseage_clusters(test)

NameError: name 'kmeans' is not defined

In [ ]:
plt.figure(figsize=(20,10))
sns.set(style="darkgrid")

plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm, 
            alpha=.5, color="mediumblue", s=100, label="Model: LinearRegression")
m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_lm, 1)
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='limegreen', label='Line of Regrssion', linewidth=5)
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="yellow", label='Baseline', linewidth=5)
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='The Ideal Line: Predicted = Actual', linewidth=5)
plt.title('Model: LinearRegression')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
plt.figure(figsize=(20,10))
sns.set(style="darkgrid")

plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm-y_validate.logerror, 
            alpha=.5, color="mediumblue", s=100, label="Model: LinearRegression")
plt.axhline(label="No Error", color='black', linewidth=7)
plt.title('Model: LinearRegression')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
plt.subplots(3, 5, figsize=(8,16), sharey=True)
sns.set(style="darkgrid")
plt.title("Comparing the Distribution of logerrors to Distributions of Predicted logerrors Linear Regression Models")
plt.xlabel("logerror", size = 15)
plt.ylabel("logerror Count", size = 15)

plt.subplot(3,1,1)
plt.hist(y_validate.logerror, color='cyan', alpha=.5,  ec='black')
plt.title('Actual logerrors', size=15)

plt.subplot(3,1,2)
plt.hist(y_validate.logerror_pred_lm, color='lawngreen', alpha=.5,  ec='black')
plt.title('Model: LinearRegression', size=15)

plt.subplot(3,1,3)
plt.hist(y_validate.logerror, color='lawngreen', alpha=.5, label="Actual Final logerrors", ec='black')
plt.hist(y_validate.logerror_pred_lm, color='cyan', alpha=.5, label="Model: LinearRegression", ec='black')
plt.title("All Graphs Stacked", size=15)
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
lm = LinearRegression(normalize=True)
lm.fit(X_test, y_test.logerror)
y_test['logerror_pred_lm'] = lm.predict(X_test)
rmse_test_lm = mean_squared_error(y_test.logerror, y_test.logerror_pred_lm)**(1/2)
print("RMSE for OLS using LinearRegression Test Data: ", rmse_test_lm)

In [ ]:
y_test

In [ ]:
r2_score_baseline = r2_score(y_validate.logerror, y_validate.logerror_pred_median)
r2_score_baseline

In [ ]:
r2_score_ols = r2_score(y_validate.logerror, y_validate.logerror_pred_lm)
r2_score_ols

In [ ]:
X_train, X_validate, X_test, y_train, y_validate, y_test = wrangle.split_train_validate_test(train, validate, test)

# Baseline

In [ ]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
    # turn it into a single pandas dataframe
y_validate = pd.DataFrame(y_validate)
    # wrap them as dataframes

# 1. Predict logerror_pred_mean
    # 2 different aselines of mean and medium
logerror_pred_mean = y_train['logerror'].mean()
y_train['logerror_pred_mean'] = logerror_pred_mean
y_validate['logerror_pred_mean'] = logerror_pred_mean

# 2. compute logerror_pred_median
    # same process as mean (above)
logerror_pred_median = y_train['logerror'].median()
y_train['logerror_pred_median'] = logerror_pred_median
y_validate['logerror_pred_median'] = logerror_pred_median

# 3. RMSE of logerror_pred_mean
rmse_train_mean = mean_squared_error(y_train.logerror, 
                                     y_train.logerror_pred_mean)**(1/2)
    # stick with root mean square error
        # not your only option but that is what we will be using here
            # just because it is eaiest to us and explain
    # remember when you call you it will be your y_true and y_pred
rmse_validate_mean = mean_squared_error(y_validate.logerror, 
                                        y_validate.logerror_pred_mean)**(1/2)
    # do the same thing for the validate set as done above for the train set
    
print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train_mean, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_mean, 2))
print(' ')
# 4. RMSE of logerror_pred_median
rmse_train_medium = mean_squared_error(y_train.logerror, 
                                       y_train.logerror_pred_median)**(1/2)

rmse_validate_medium = mean_squared_error(y_validate.logerror, 
                                          y_validate.logerror_pred_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train_medium, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_medium, 2))

# median comes out a little higher than mean 
    # but a little lower than our validation set

# OLS Model

In [ ]:
# create the model object
lm = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm.fit(X_train, y_train.logerror)
    # just call y_train.actual_target

# predict train
y_train['logerror_pred_lm'] = lm.predict(X_train)

# evaluate: rmse
rmse_train_lm = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm)**(1/2)

# predict validate
y_validate['logerror_pred_lm'] = lm.predict(X_validate)

# evaluate: rmse
rmse_validate_lm = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm)**(1/2)
    # make sure you are using x_validate an not x_train

print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train_lm, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lm)

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='_nolegend_')
plt.annotate("Baseline: Predict Using Median", (16, 9.5))
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="black", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm, 
            alpha=.5, color="blue", s=100, label="Model: LinearRegression")
plt.legend()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

In [ ]:
# create the model object
lars = LassoLars(alpha=1.0)
    # what alpha is in this case
        # kidn of that regualizer that make lasso a little bit different
        # can go from 0 to infinity
            # it will eventually stop changing the efficivicy of you model 
                # depenind on th enumber of features being fed in
            # if alpha = 0 it is the same thing as a los model

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lars.fit(X_train, y_train.logerror)
    # fit the thing

# predict train
y_train['logerror_pred_lars'] = lars.predict(X_train)

# evaluate: rmse
rmse_train_lars = median_squared_error(y_train.logerror, y_train.logerror_pred_lars)**1/2

# predict validate
y_validate['logerror_pred_lars'] = lars.predict(X_validate)

# evaluate: rmse
rmse_validate_lars = median_squared_error(y_validate.logerror, y_validate.logerror_pred_lars)**1/2

print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train_lars, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lars)

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='_nolegend_')
plt.annotate("Baseline: Predict Using Median", (16, 9.5))
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="black", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_validate.logerror, y_validate.logerror_pred_lars, 
            alpha=.5, color="darkred", s=100, label="Model: LinearRegression")

plt.legend()
plt.xlabel("Actual Final Grade")
plt.ylabel("Predicted Final Grade")
plt.title("Where are predictions more extreme? More modest?")

# Tweedie

In [ ]:
# create the model object
glm = TweedieRegressor(power=1, alpha=0)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
glm.fit(X_train, y_train.logerror)

# predict train
y_train['logerror_pred_glm'] = glm.predict(X_train)

# evaluate: rmse
rmse_train_glm = mean_squared_error(y_train.logerror, y_train.logerror_pred_glm)**1/2

# predict validate
y_validate['logerror_pred_glm'] = glm.predict(X_validate)

# evaluate: rmse
rmse_validate_glm = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_glm)**1/2

print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train_glm, 
      "\nValidation/Out-of-Sample: ", rmse_validate_glm)

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.logerror, y_validate.logerror_pred_mean, alpha=.5, color="black", label='_nolegend_')
plt.annotate("Baseline: Predict Using Median", (16, 9.5))
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="black", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_validate.logerror, y_validate.logerror_pred_glm, 
            alpha=.5, color="darkviolet", s=100, label="Model: TweedieRegressor")

plt.legend()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

# Polynomial Regression degree 2

In [ ]:
# make the polynomial features to get a new set of features
pf = PolynomialFeatures(degree=2)

# fit and transform X_train_scaled
X_train_degree2 = pf.fit_transform(X_train)

# transform X_validate_scaled & X_test_scaled
X_validate_degree2 = pf.transform(X_validate)
    # dont call fit transform just call transform
        # we only fit train not validate
X_test_degree2 = pf.transform(X_test)

In [ ]:
# create the model object
lm2 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm2.fit(X_train_degree2, y_train.logerror)

# predict train
y_train['logerror_pred_lm2'] = lm2.predict(X_train_degree2)

# evaluate: rmse
rmse_train_lm2 = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm2)**1/2

# predict validate
y_validate['logerror_pred_lm2'] = lm2.predict(X_validate_degree2)

# evaluate: rmse
rmse_validate_lm2 = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm2)**1/2

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train_lm2, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lm2)

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='_nolegend_')
plt.annotate("Baseline: Predict Using Median", (16, 9.5))
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="black", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm2, 
            alpha=.5, color="darkturquoise", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

# Polynomial Regression degree 3

In [ ]:
# make the polynomial features to get a new set of features
pf3 = PolynomialFeatures(degree=3)

# fit and transform X_train_scaled
X_train_degree3 = pf3.fit_transform(X_train)

# transform X_validate_scaled & X_test_scaled
X_validate_degree3 = pf3.transform(X_validate)
    # dont call fit transform just call transform
        # we only fit train not validate
X_test_degree3 = pf3.transform(X_test)

In [ ]:
# create the model object
lm3 = LinearRegression(normalize=True)

# fit the model to our training data. We must specify the column in y_train, 
# since we have converted it to a dataframe from a series! 
lm3.fit(X_train_degree3, y_train.logerror)

# predict train
y_train['logerror_pred_lm3'] = lm3.predict(X_train_degree2)

# evaluate: rmse
rmse_train_lm3 = mean_squared_error(y_train.logerror, y_train.logerror_pred_lm2)**1/2

# predict validate
y_validate['logerror_pred_lm3'] = lm3.predict(X_validate_degree3)

# evaluate: rmse
rmse_validate_lm3 = mean_squared_error(y_validate.logerror, y_validate.logerror_pred_lm2)**1/2

print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train_lm3, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lm3)

In [ ]:
# y_validate.head()
plt.figure(figsize=(16,8))
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='_nolegend_')
plt.annotate("Baseline: Predict Using Median", (16, 9.5))
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="black", label='_nolegend_')
plt.annotate("The Ideal Line: Predicted = Actual", (.5, 3.5), rotation=15.5)

plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm3, 
            alpha=.5, color="darkturquoise", s=100, label="Model 2nd degree Polynomial")
plt.legend()
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Where are predictions more extreme? More modest?")
# plt.annotate("The polynomial model appears to overreact to noise", (2.0, -10))
# plt.annotate("The OLS model (LinearRegression)\n appears to be most consistent", (15.5, 3))
plt.show()

In [ ]:
plt.subplots(6, 1, figsize=(8,22), sharey=True)
sns.set(style="darkgrid")

plt.subplot(6,1,1)
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm, 
            alpha=.5, color="mediumblue", s=100, label="Model: LinearRegression")
m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_lm, 1)
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='fuchsia', label='Line of Regrssion')
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regession')
plt.title('Model: LinearRegression')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(6,1,2)
plt.scatter(y_validate.logerror, y_validate.logerror_pred_glm, 
            alpha=.5, color="teal", s=100, label="Model: TweedieRegressor")
m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_glm, 1) 
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='fuchsia', label='Line of Regrssion')
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regression')
plt.title('Model: TweedieRegressor')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(6,1,3)
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm, 
            alpha=.5, color="limegreen", s=100, label="Model 2nd degree Polynomial")

m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_lm2, 1) 
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='fuchsia', label='Line of Regrssion')

plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regression')
plt.title('Model 2nd degree Polynomial')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.subplot(5,1,3)
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm3, 
            alpha=.5, color="limegreen", s=100, label="Model 3rd degree Polynomial")

m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_lm3, 1) 
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='fuchsia', label='Line of Regrssion')

plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regression')
plt.title('Model 2nd degree Polynomial')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')


plt.subplot(6,1,4)
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lars, 
            alpha=.5, color="yellow", s=100, label="Model: Lasso")

m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_lars, 1) 
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='fuchsia', label='Line of Regrssion')

plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regression')
plt.title('Model: Lasso')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')


plt.subplot(6,1,5)
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lars, 
            alpha=.5, color="yellow", s=100, label="Model: Lasso")

m, b = np.polyfit(y_validate.logerror, y_validate.logerror_pred_lars, 1) 
plt.plot(y_validate.logerror, m*y_validate.logerror+b, color='fuchsia', label='Line of Regrssion')

plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regression')
plt.title('Model: Lasso')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')



plt.subplot(6,1,6)
plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='Baseline')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="cyan", label='OLS Regression')
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm, 
            alpha=.5, color="mediumblue", s=100, label="Model: LinearRegression")
plt.scatter(y_validate.logerror, y_validate.logerror_pred_glm, 
            alpha=.5, color="teal", s=100, label="Model: TweedieRegressor")
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lm2, 
            alpha=.5, color="yellow", s=100, label="Model 2nd degree Polynomial")
plt.scatter(y_validate.logerror, y_validate.logerror_pred_lars, 
            alpha=.5, color="limegreen", s=100, label="Model: Lasso")

plt.plot(y_validate.logerror, y_validate.logerror_pred_median, alpha=.5, color="black", label='_nolegend_')
plt.plot(y_validate.logerror, y_validate.logerror, alpha=.5, color="black", label='_nolegend_')
plt.title('All Models stacked')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')


In [ ]:
print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train_mean, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_mean, 2))
print("--------------------------------------------------------------")
print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train_medium, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_medium, 2))
print("--------------------------------------------------------------")
print("RMSE for OLS using LinearRegression\nTraining/In-Sample: ", rmse_train_lm, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lm)
print("--------------------------------------------------------------")
print("RMSE for Lasso + Lars\nTraining/In-Sample: ", rmse_train_lars, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lars)
print("--------------------------------------------------------------")
print("RMSE for GLM using Tweedie, power=1 & alpha=0\nTraining/In-Sample: ", rmse_train_glm, 
      "\nValidation/Out-of-Sample: ", rmse_validate_glm)
print("--------------------------------------------------------------")
print("RMSE for Polynomial Model, degrees=2\nTraining/In-Sample: ", rmse_train_lm2, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lm2)
print("--------------------------------------------------------------")
print("RMSE for Polynomial Model, degrees=3\nTraining/In-Sample: ", rmse_train_lm3, 
      "\nValidation/Out-of-Sample: ", rmse_validate_lm3)